In [ ]:
import boto3
import os
root_s3 = 'images'
root = r'C:\Users\nisae\OneDrive\Documents\DataScientist\P8_Poitier_Nicolas\archive\fruits-360_dataset\fruits-360\Training'
region_name="eu-west-3"

# Suppression des Espaces dans le nom des dossiers et fichiers

In [ ]:
list_to_rename = []
list_new_name = []

for path, subdirs, files in os.walk(root):
    #print()
    #print("chemin",path)
    #print("subdir")
    for d in subdirs:        
        nd = d.replace(' ','_')
        if d != nd:
            list_to_rename.append(path.replace(' ','_')+'\\'+d)
            list_new_name.append(path.replace(' ','_')+'\\'+nd)
            
        #print(d,nd)
    #print("files")
    for f in files:
        nf = f.replace(' ','_')
        if f != nf:
            list_to_rename.append(path.replace(' ','_')+'\\'+f)
            list_new_name.append(path.replace(' ','_')+'\\'+nf)
        #print(f,nf)

for i in range(len(list_to_rename)):
    os.rename(list_to_rename[i],list_new_name[i])

# Chargement des données dans AWS S3

In [ ]:
def load_aws_key():
    with open('keys.txt','r') as f:
        msg = f.read()
    ID = str(msg).split('\n')[0]
    KEY = msg.split('\n')[1]
    os.environ["AWS_ACCESS_KEY_ID"]=ID
    os.environ["AWS_SECRET_ACCESS_KEY"]=KEY
    return ID, KEY
load_aws_key()

session = boto3.session.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
                                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3_client = session.client(service_name='s3', region_name=region_name)

#Création du dossier directory_name
directory_name = root_s3
s3_client.put_object(Bucket=bucket_name, Key=(directory_name+'/'))
s3 = boto3.resource("s3", region_name=region_name)

In [ ]:
for path, subdirs, files in os.walk(root):
    #print()
    #print("subdir")
    for d in subdirs:        
        directory_name = (path.replace(root,root_s3)+'\\'+d+'\\').replace('\\','/')
        #print(directory_name)
        s3_client.put_object(Bucket=bucket_name, Key=(directory_name))
    #print()
    #print("files")
    for f in files:        
        file = (path+'\\'+f).replace('\\','/').replace('\\','/')
        distant_file = (path.replace(root,root_s3)+'\\'+f).replace('\\','/')
        #print(file,distant_file)
        s3.Object(bucket_name, distant_file ).upload_file(file)